In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#WORD EMBEDDİNGS AND SENTİMENTS

* Kelime Gömme Yapılacak
* Duygu Tahmini için sinir ağı eğitilecek
* Her duygunun nasıl göründüğünü görselleştirilecek

##1-İmporting Tensorflow libraries

In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
# İmporting numpy and pandas libraries
import numpy as np
import pandas as pd

## 2- Get The Dataset
  * Amazon ve Yelp den duyguları içeren bir veri seti içeri aktaracaz
  * 1 olumlu 0 olumsuz duyguyu ifade eder

In [2]:
!wget --no-check-certificate \
    -O /tmp/sentiment.csv https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P

--2023-03-07 00:06:43--  https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving drive.google.com (drive.google.com)... 173.194.197.102, 173.194.197.101, 173.194.197.100, ...
Connecting to drive.google.com (drive.google.com)|173.194.197.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i62aob8psvimte3dnaj7le0c0crb5487/1678147575000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P?uuid=87c9480e-1bd7-4945-84b4-170079f84891 [following]
--2023-03-07 00:06:43--  https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i62aob8psvimte3dnaj7le0c0crb5487/1678147575000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P?uuid=87c9480e-1bd7-4945-84b4-170079f84891
Resolving doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)... 74.125.69.132, 2607:f8b0:4001:c08::84
Connecting to doc

In [4]:
dataset = pd.read_csv("/tmp/sentiment.csv")

In [11]:
#AYIKLAMA İŞLEMİ
# Cümleleri içeren bir liste seti oluşturalım
sentences = dataset['text'].tolist()

# Duyguları içeren bir liste oluşturalım
labels = dataset['sentiment'].tolist()

print(sentences[0:5])
print("*"*40)
print(labels[0:5])

['So there is no way for me to plug it in here in the US unless I go by a converter.', 'Good case Excellent value.', 'Great for the jawbone.', 'Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!', 'The mic is great.']
****************************************
[0, 1, 1, 0, 1]


##3-Let's Create Training And Test Sets

In [14]:
# Öncelikle train ve test setimizin kesilme uzunluğunu belirleyelim
# train set 0.8      test set = 0.2 yapalım
training_size = int(len(sentences)*0.8)

# training size a göre cümleleri ve duyguları train_test setlerimizi oluşturalım
training_sentences = sentences[:training_size]
testing_sentences = sentences[training_size :]

training_labels = labels[:training_size]
testing_labels = labels[training_size :]

# Etiketleri ağda sonra kullanmak üzere numpy dizisi oluşturalım
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

##4-Tokenize-Padding Train And Test DataSet

In [16]:
from os import truncate
#PARAMETRELER
vocab_size = 1000   # kelimelere verilecek sayıların boyutu
embedding_dim = 16   # gömme boyutu(Bir kelimenin temsil edebileceği boyut)
max_length = 100      # cümlelerin max uzunluğu (100 den fazla olan kesilecek)
trunc_type = 'post'   # Kesilecek cümle sondan kesilsin(Belirtmezsek varsayılan baştan keser)
padding_type = 'post'  # 100 kelimeden küçük olan kelimelerin doldururken arkasına ekle demek istiyoruz
oov_tok = "<OOV>"   # Tanımadğı kelimeleri temsil etmesi için sabir bir belirteç oluşturuyoruz

# tokenizer instance oluşturmakla başlayalım
tokenizer = Tokenizer(num_words =vocab_size,oov_token = oov_tok)

# kelimeleri diziye çevirelim
tokenizer.fit_on_texts(training_sentences)

# Oluşturduğumuz diziyi sözlük halinde alalım
word_index = tokenizer.word_index

# *****************************************************

#Cümlelerimizi diziye çevirelim
sequences = tokenizer.texts_to_sequences(training_sentences)

# Dizimizi kesme ve padding kullanarak birbirlerine denkleyelim
padded = pad_sequences(sequences,maxlen = max_length,padding = padding_type , truncating=trunc_type)


# **********************************************************
# Test Cümlelerini diziye çevirelim
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)

# Denkleme işlemini yapalım kesme padding uzunluk işlemleri
testing_padded = pad_sequences(testing_sequences,maxlen = max_length,padding = padding_type,truncating=trunc_type)


##5-Review a Sequences

In [17]:
# Herşeyin uygun olup olmadığını kontrol edelim

reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i,'?') for i in text])

print(decode_review(padded[1]))
print(decode_review(training_sentences[1]))

good case excellent value ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?


##6-Train a Basic Sentiment Model with Embeddings

In [18]:
#Modelimizi eğitme işlemi için;
# Bir ağ oluşturacaz
# İlk Katman Gömme katmanı
# Çıkış katmanı 0 yada 1 olacağından Tek çıkış olacak
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size , embedding_dim,input_length=max_length), # Gömme işlemi için kelimelerin boyutu,gömme boyutu ve girişte cümlelerin max uzunluğu
    tf.keras.layers.Flatten(),  # Düzleştirme işlemi yapıyoruz
    tf.keras.layers.Dense(6,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")

])

model.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ["accuracy"])  # binary_crossentropy(0-1 için)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           16000     
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 6)                 9606      
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 25,613
Trainable params: 25,613
Non-trainable params: 0
_________________________________________________________________


In [19]:
num_epochs = 10
model.fit(padded,training_labels_final,epochs = num_epochs,validation_data = (testing_padded,testing_labels_final))

Epoch 1/10
50/50 [==============================] - 11s 115ms/step - loss: 0.6938 - accuracy: 0.5229 - val_loss: 0.6964 - val_accuracy: 0.4110
Epoch 2/10
50/50 [==============================] - 2s 39ms/step - loss: 0.6872 - accuracy: 0.5229 - val_loss: 0.7005 - val_accuracy: 0.4110
Epoch 3/10
50/50 [==============================] - 2s 34ms/step - loss: 0.6739 - accuracy: 0.5229 - val_loss: 0.6976 - val_accuracy: 0.4110
Epoch 4/10
50/50 [==============================] - 1s 29ms/step - loss: 0.6417 - accuracy: 0.6265 - val_loss: 0.6874 - val_accuracy: 0.4687
Epoch 5/10
50/50 [==============================] - 1s 21ms/step - loss: 0.5856 - accuracy: 0.6836 - val_loss: 0.6554 - val_accuracy: 0.6165
Epoch 6/10
50/50 [==============================] - 1s 15ms/step - loss: 0.5210 - accuracy: 0.8060 - val_loss: 0.6588 - val_accuracy: 0.5614
Epoch 7/10
50/50 [==============================] - 1s 16ms/step - loss: 0.4638 - accuracy: 0.8544 - val_loss: 0.6380 - val_accuracy: 0.6140
Epoch 8/10


##7-Get files for visualizing the network
* Bu işlemi oluşturmuş olduğumuz dizileri görselleştirmek için kullanacağız
* İşlem sonunda tsv uzantılı bir dosya elde edeceğiz
* Elde ettiğimiz bu dosyayı aşağıdaki adreste load butonunu kullanarak görselleştirme işlemi yapacaz
* http://projector.tensorflow.org/

In [25]:
# Önce gömme katmanının ağırlıklarını alalım
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)  # shape : (vocab_size , embeddings_dim)

(1000, 16)


In [27]:
# Gömülü vektörleri ve meta verileri yazalım
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [28]:
# Download the files
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##8- Predicting Sentiment in New Reviews
* Hiç Görmediği cümlelerin duygularını tahmin edelim


In [29]:
# Cümlelerimizi tanımlayalım
fake_reviews = ['I love this phone', 'I hate spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head']

print(fake_reviews) 


['I love this phone', 'I hate spaghetti', 'Everything was cold', 'Everything was hot exactly as I wanted', 'Everything was green', 'the host seated us immediately', 'they gave us free chocolate cake', 'not sure about the wilted flowers on the table', 'only works when I stand on tippy toes', 'does not work when I stand on my head']


In [30]:
# Diziye çevirme işlemi yapalım
padding_type = 'post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)

# padding - truncate ve max length işlemlerini yapalım
sample_padded = pad_sequences(sample_sequences,padding=padding_type,maxlen = max_length)

# Tahmine başlayalım 
classes = model.predict(sample_padded)

# Sonuçlar için bir for döngüsü oluşturalım 
# 1 e ne kadar yakınsa olumlu olma ihtimali o kadar yüksektir
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')

1/1 [==============================] - 0s 94ms/step
I love this phone
[0.9655568]


I hate spaghetti
[0.27540037]


Everything was cold
[0.5053474]


Everything was hot exactly as I wanted
[0.64136773]


Everything was green
[0.51515454]


the host seated us immediately
[0.73586786]


they gave us free chocolate cake
[0.80141467]


not sure about the wilted flowers on the table
[0.2647199]


only works when I stand on tippy toes
[0.9228611]


does not work when I stand on my head
[0.2654422]


